In [1]:
import pandas as pd
import altair as alt

path = 'data/2014-2020.xlsx'
df = pd.read_excel(path)
df.drop(df.columns[len(df.columns) - 1], axis=1, inplace=True)
df.columns = ['time_date', 'AES', 'TEC', 'VDE', 'TES', 'GES', 'GAES_GEN', 'consumption', 'GAES_PUMP', 'UK_BLR_RUS',
              'UK_EURO', 'UK_MLD']
df.head()


,time_date,AES,TEC,VDE,TES,GES,GAES_GEN,consumption,GAES_PUMP,UK_BLR_RUS,UK_EURO,UK_MLD
0,24-31.12.2020,9235,2039,621,4942,385.0,0,16693,-405.0,-84.0,-11.0,-29.0
1,23-31.12.2020,9221,2159,707,5549,470.0,0,17805,0.0,-43.0,-212.0,-46.0
2,22-31.12.2020,9249,2377,709,5906,1000.0,0,18870,0.0,-13.0,-328.0,-30.0
3,21-31.12.2020,9256,2499,702,6329,909.0,322,19887,0.0,0.0,-64.0,-66.0
4,20-31.12.2020,9213,2521,702,6640,823.0,602,20387,0.0,-29.0,-48.0,-37.0


In [2]:
df_1 = df.copy(deep=False)


def label_year(row):
    if row['time_date'].endswith('2020'):
        return '2020'
    if row['time_date'].endswith('2019'):
        return '2019'
    if row['time_date'].endswith('2018'):
        return '2018'
    if row['time_date'].endswith('2017'):
        return '2017'
    if row['time_date'].endswith('2016'):
        return '2016'
    if row['time_date'].endswith('2015'):
        return '2015'
    if row['time_date'].endswith('2014'):
        return '2014'
    return '0'


df_1['year'] = df_1.apply(lambda row: label_year(row), axis=1)
df_to_draw = df_1.copy(deep=False)
df_to_draw.drop(['consumption', 'GAES_PUMP'], axis=1, inplace=True)
df_to_draw = df_to_draw.groupby('year')[['AES', 'TEC', 'VDE', 'TES', 'GES', 'GAES_GEN']].sum()

In [3]:
df_to_draw.head()


,AES,TEC,VDE,TES,GES,GAES_GEN
year,,,,,,
2014,88204418,14684731,1606386,68605877,7991074.0,839842
2015,87413763,12041074,1234936,50259819,4964478.0,1554857
2016,80762094,12841033,1093190,49879543,7113989.0,1621377
2017,85314258,12208062,1177032,44945303,8531952.0,1575099
2018,84351328,12261018,1863726,47747092,10072767.0,1564867


In [4]:
df_to_draw['year'] = [x for x in range(2014, 2021)]
df_to_draw = pd.melt(df_to_draw, id_vars=['year'], var_name = 'corporation', value_name = 'consumption')
df_to_draw = df_to_draw.sort_values(by=['consumption'])
df_to_draw.head()

,year,corporation,consumption
35,2014,GAES_GEN,839842.0
16,2016,VDE,1093190.0
17,2017,VDE,1177032.0
15,2015,VDE,1234936.0
40,2019,GAES_GEN,1323192.0


# № 1

Вибрав для представлення стовпці з посортованими по зростанню використання електороенергії способа.
Переваги: можливо миттєво побачити що виробляє більше, а що менше.
Недоліки: важко порівнювати джерела, які виробляють незначну кількість


In [ ]:
alt.Chart(df_to_draw).mark_bar().encode(
    x = alt.X('year:N'),
    y = alt.Y('consumption:Q'),
    color = alt.Color('corporation:N'),
    order=alt.Order(
      # Sort the segments of the bars by this field
      'consumption',
      sort='descending'
    )
).properties(width = 500, height = 500)


# № 2

Переваги: чітко видно зміну, видно різницю загальної кількості між роками, замітна різниця споживання одного джерела.
Недоліки: важко витягнути чітке число споживання енерегії з одного джерела.

In [ ]:
alt.Chart(df_to_draw).mark_area().encode(
    x = alt.X('year:T'),
    y = alt.Y('consumption:Q'),
    color=alt.Color("corporation:N")
)




# №3
Переваги: чітко замітні просадки кожної джерела
Недоліки: малі значення доволі сильно зливаються

In [ ]:
def hour(row):
    ind = row['time_date'].find("-")
    return row['time_date'][:ind]

df_3 = df.copy(deep=False)
df_3['hour'] = df_3.apply(lambda row: hour(row), axis=1)
df_3.drop(['consumption', 'GAES_PUMP', 'time_date'], axis = 1, inplace = True)
df_3 = df_3.groupby('hour')[['AES', 'TEC', 'VDE', 'TES', 'GES', 'GAES_GEN']].mean()
df_3['hour'] = [x for x in range(1, 25)]
df_3 = pd.melt(df_3, id_vars=['hour'], var_name = 'corporation', value_name = 'consumption')

alt.Chart(df_3).transform_filter(alt.FieldRangePredicate(field='hour', range=[1, 24])).mark_line().encode(
    x = alt.X('hour:Q', scale=alt.Scale(domain=[1, 24])),
    y = alt.Y('consumption:Q'),
    color = alt.Color('corporation:N')
).properties(height = 150, width = 700)


